In [1]:
%load_ext lab_black

import os
import pandas as pd
from PIL import Image
import xml.etree.cElementTree as ET
import lxml.etree as le
from shutil import copyfile, rmtree
import sys

BASE_PATH = os.path.join("/tf/notebooks/CEIA/computer-vision-2/tp-final")
sys.path.insert(0, BASE_PATH)

from utils import load_image, get_filename_generator

In [2]:
BASE_DS_PATH = "/tf/datasets"

FINAL_DS_PATH = os.path.join(BASE_DS_PATH, "final-dataset")
FINAL_DS_IMAGES_PATH = os.path.join(FINAL_DS_PATH, "images")


DS_PATHS = {
    "mercosur": os.path.join(BASE_DS_PATH, "artificial-mercosur-license-plates")
}

filename_generator = get_filename_generator()

Preprocesamiento de [Artificial lercosur license plates](https://data.mendeley.com/datasets/nx9xbs4rgx/2)

In [3]:
MERCOSUR_PATHS = {
    "dataset": os.path.join(DS_PATHS["mercosur"], "dataset.csv"),
    "labels": os.path.join(DS_PATHS["mercosur"], "labels"),
    "images": os.path.join(DS_PATHS["mercosur"], "images"),
}

mercosur_df = pd.read_csv(MERCOSUR_PATHS["dataset"], index_col=0)
mercosur_df

,image,label,class,x_center,y_center,width,height
0,monitoring_system_4.JPG,monitoring_system_4.txt,0,0.180625,0.650833,0.128750,0.058333
1,monitoring_system_1817.JPG,monitoring_system_1817.txt,0,0.747500,0.034167,0.072500,0.031667
2,monitoring_system_1864.JPG,monitoring_system_1864.txt,0,0.714375,0.152500,0.096250,0.041667
3,monitoring_system_2484.JPG,monitoring_system_2484.txt,0,0.632500,0.625833,0.127500,0.058333
4,monitoring_system_2066.JPG,monitoring_system_2066.txt,0,0.885000,0.085000,0.087500,0.040000
...,...,...,...,...,...,...,...
3835,monitoring_system_2793.JPG,monitoring_system_2793.txt,0,0.616875,0.830833,0.163750,0.071667
3836,monitoring_system_264.JPG,monitoring_system_264.txt,0,0.801250,0.102500,0.102500,0.038333
3837,monitoring_system_2718.JPG,monitoring_system_2718.txt,0,0.925625,0.087500,0.083750,0.035000
3838,parking_lot1_35.JPG,parking_lot1_35.txt,0,0.388940,0.544118,0.057904,0.046841


In [4]:
files_annotations = []

for i in range(len(mercosur_df)):
    # Image and label files
    try:
        image_filename = mercosur_df.iloc[i]["image"]
        image_label = mercosur_df.iloc[i]["label"]
        with open(
            os.path.join(MERCOSUR_PATHS["labels"], image_label), "r"
        ) as txt_label:
            len_txt_label = txt_label.read().count("\n")

        # Only save cropped images with one vehicle
        if "cropped" in image_label and len_txt_label == 1:
            # New filename
            new_filename = next(filename_generator)

            # Open the image
            img = load_image(MERCOSUR_PATHS["images"], image_filename)
            (height, width, _) = img.shape

            # Get box info
            box_width = int(mercosur_df.iloc[i]["width"] * width)
            box_height = int(mercosur_df.iloc[i]["height"] * height)
            x_center = int(mercosur_df.iloc[i]["x_center"] * width)
            x_min = int(x_center - box_width / 2)
            x_max = int(x_center + box_width / 2)
            y_center = int(mercosur_df.iloc[i]["y_center"] * height)
            y_min = int(y_center - box_height / 2)
            y_max = int(y_center + box_height / 2)

            # Save new image
            cv2.imwrite(os.path.join(FINAL_DS_IMAGES_PATH, new_filename), img)

            # Save info to generate .csv file
            files_annotations.append((new_filename, x_min, y_min, x_max, y_max))
    except:
        pass

In [5]:
img = load_image(MERCOSUR_PATHS["images"], "cropped_parking_lot_1.JPG")
cv2.imwrite(os.path.join(FINAL_DS_IMAGES_PATH, "test.jpg"), img)

True